In [1]:
# Install geopy and folium library
!conda install -c conda-forge geopy --yes 
!conda install -c conda-forge folium=0.5.0 --yes

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.18.1-py_0 conda-forge

geographiclib- 100% |################################| Time: 0:00:00  24.40 MB/s
geopy-1.18.1-p 100% |################################| Time: 0:00:00  36.57 MB/s
Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  53.94 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  34.66 MB/s
vincent-0.4.4- 100% |###################

In [2]:
# Import the libraries
import folium
from geopy.geocoders import Nominatim
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup

In [3]:
# The code was removed by Watson Studio for sharing.

,city,lat,lng,country,iso2,admin,capital,population,population_proper
0,Toronto,43.666667,-79.416667,Canada,CA,Ontario,admin,5213000,3934421
1,Montreal,45.500000,-73.583333,Canada,CA,Quebec,NaN,3678000,2356556
2,Vancouver,49.250000,-123.133333,Canada,CA,British Columbia,NaN,2313328,603502
3,Ottawa,45.416667,-75.700000,Canada,CA,Ontario,primary,1145000,812129
4,Calgary,51.083333,-114.083333,Canada,CA,Alberta,NaN,1110000,915322


In [4]:
# Drop iso2, captial and population_proper
df.drop(['capital', 'population_proper'], axis=1, inplace=True)

In [5]:
# Rename admin to Province
df.rename(columns={'admin':'province'}, inplace=True)
df.head()

,city,lat,lng,country,iso2,province,population
0,Toronto,43.666667,-79.416667,Canada,CA,Ontario,5213000
1,Montreal,45.500000,-73.583333,Canada,CA,Quebec,3678000
2,Vancouver,49.250000,-123.133333,Canada,CA,British Columbia,2313328
3,Ottawa,45.416667,-75.700000,Canada,CA,Ontario,1145000
4,Calgary,51.083333,-114.083333,Canada,CA,Alberta,1110000


In [6]:
# Get the latitude and longitude for Canada
country = str(df.country[0])
country_code = str(df.iso2[0])
address = country + "," + country_code

geolocator = Nominatim()
location = geolocator.geocode(address)
ca_lat = location.latitude
ca_lng = location.longitude
print ('Latitude & Longitude of Canada:', ca_lat, ca_lng)

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:6: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.


Latitude & Longitude of Canada: 61.0666922 -107.9917071


In [7]:
# Generate map around Canada
map_ca = folium.Map(location=[ca_lat, ca_lng], zoom_start=4)
map_ca

In [8]:
# import the source code and get the land area to calculate the population density for each province
source_code = requests.get('https://en.wikipedia.org/wiki/List_of_Canadian_provinces_and_territories_by_area').text
soup = BeautifulSoup(source_code, 'lxml')
table = soup.find_all('table', attrs={'class':'wikitable sortable'})[1]

province_ca = []
land_area = []
prov_lat = []
prov_lng = []
for content in table.find_all('tr')[1:]:
    province_ca.append((content.find_all('td')[1].text).strip())
    land_area.append((content.find_all('td')[2].text).strip())

# Convert the landarea to float value
flt_land_area = [x.replace(',', '') for x in land_area]
print (province_ca)
print (flt_land_area)

['Nunavut', 'Quebec', 'Northwest Territories', 'British Columbia', 'Ontario', 'Alberta', 'Saskatchewan', 'Manitoba', 'Yukon', 'Newfoundland and Labrador', 'New Brunswick', 'Nova Scotia', 'Prince Edward Island', 'Canada']
['1936113', '1365128', '1183085', '925186', '917741', '642317', '591670', '553556', '474391', '373872', '71450', '53338', '5660', '9093507']


In [9]:
# get the latitude and longitude for the province
geolocator = Nominatim()
for province in province_ca:
    location = geolocator.geocode(province)
    prov_lat.append(location.latitude)
    prov_lng.append(location.longitude)
    
print (prov_lat, prov_lng)

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  from ipykernel import kernelapp as app


[70.0000073, 52.4760892, 65.0000004, 55.001251, 50.000678, 55.001251, 55.5321257, 55.001251, 63.000147, 53.8217327, 46.500283, 45.1960403, 46.3355508, 61.0666922] [-90.0, -71.8258668, -118.0, -125.002441, -86.000977, -115.002136, -106.1412243, -97.001038, -136.002502, -61.2295532, -66.750183, -63.1653789, -63.1466676, -107.9917071]


In [10]:
# convert the above list to pandas dataframe
land_Area_ca = {'province': province_ca, 'Land Area(km Sq)': flt_land_area, 'Latitude': prov_lat, 'Longitude': prov_lng}
col_labels = ['province', 'Land Area(km Sq)', 'Latitude', 'Longitude']
df_landarea_ca = pd.DataFrame(land_Area_ca, columns=col_labels)
df_landarea_ca = df_landarea_ca.drop([13])
df_landarea_ca

,province,Land Area(km Sq),Latitude,Longitude
0,Nunavut,1936113,70.000007,-90.000000
1,Quebec,1365128,52.476089,-71.825867
2,Northwest Territories,1183085,65.000000,-118.000000
3,British Columbia,925186,55.001251,-125.002441
4,Ontario,917741,50.000678,-86.000977
5,Alberta,642317,55.001251,-115.002136
6,Saskatchewan,591670,55.532126,-106.141224
7,Manitoba,553556,55.001251,-97.001038
8,Yukon,474391,63.000147,-136.002502
9,Newfoundland and Labrador,373872,53.821733,-61.229553


In [11]:
# Group by Province and sum up the population
df_sum_population = pd.DataFrame(df.groupby(['province'])['population'].sum()).reset_index()
df_sum_population

,province,population
0,Alberta,2530047
1,British Columbia,3426424
2,Manitoba,731999
3,New Brunswick,254834
4,Newfoundland and Labrador,185120
5,Northwest Territories,32623
6,Nova Scotia,456709
7,Nunavut,26766
8,Ontario,9788475
9,Prince Edward Island,42402


In [12]:
# Group by Province and combine the city
df_group_city = pd.DataFrame(df.groupby(['province'])['city'].apply(','.join)).reset_index()
df_group_city

,province,city
0,Alberta,"Calgary,Edmonton,Red Deer,Lethbridge,Medicine ..."
1,British Columbia,"Vancouver,Victoria,Abbotsford,Kelowna,Nanaimo,..."
2,Manitoba,"Winnipeg,Brandon,Thompson,Selkirk,Steinbach,Da..."
3,New Brunswick,"Moncton,Saint John,Fredericton,Edmundston,Bath..."
4,Newfoundland and Labrador,"Saint Johns,Corner Brook,Happy Valley,Stephenv..."
5,Northwest Territories,"Yellowknife,Hay River,Inuvik,Fort McPherson,No..."
6,Nova Scotia,"Halifax,Sydney,New Glasgow,Amherst,Yarmouth,An..."
7,Nunavut,"Iqaluit,Rankin Inlet,Arviat,Igloolik,Baker Lak..."
8,Ontario,"Toronto,Ottawa,Hamilton,Oshawa,Kitchener,Londo..."
9,Prince Edward Island,Charlottetown


In [13]:
# Merge three dataframe into one to get the complete canada details
df_ca = pd.merge(pd.merge(df_sum_population, df_group_city[['province','city']], on='province'), 
                     df_landarea_ca[['province', 'Land Area(km Sq)', 'Latitude', 'Longitude']], on='province')
df_ca['Land Area(km Sq)'] = df_ca['Land Area(km Sq)'].astype(dtype=np.float32)
df_ca

,province,population,city,Land Area(km Sq),Latitude,Longitude
0,Alberta,2530047,"Calgary,Edmonton,Red Deer,Lethbridge,Medicine ...",642317.0,55.001251,-115.002136
1,British Columbia,3426424,"Vancouver,Victoria,Abbotsford,Kelowna,Nanaimo,...",925186.0,55.001251,-125.002441
2,Manitoba,731999,"Winnipeg,Brandon,Thompson,Selkirk,Steinbach,Da...",553556.0,55.001251,-97.001038
3,New Brunswick,254834,"Moncton,Saint John,Fredericton,Edmundston,Bath...",71450.0,46.500283,-66.750183
4,Newfoundland and Labrador,185120,"Saint Johns,Corner Brook,Happy Valley,Stephenv...",373872.0,53.821733,-61.229553
5,Northwest Territories,32623,"Yellowknife,Hay River,Inuvik,Fort McPherson,No...",1183085.0,65.000000,-118.000000
6,Nova Scotia,456709,"Halifax,Sydney,New Glasgow,Amherst,Yarmouth,An...",53338.0,45.196040,-63.165379
7,Nunavut,26766,"Iqaluit,Rankin Inlet,Arviat,Igloolik,Baker Lak...",1936113.0,70.000007,-90.000000
8,Ontario,9788475,"Toronto,Ottawa,Hamilton,Oshawa,Kitchener,Londo...",917741.0,50.000678,-86.000977
9,Prince Edward Island,42402,Charlottetown,5660.0,46.335551,-63.146668


In [14]:
# find the population density of canada province
df_ca['population density(Per Sq km)'] = df_ca['population'] / df_ca['Land Area(km Sq)']
df_ca

,province,population,city,Land Area(km Sq),Latitude,Longitude,population density(Per Sq km)
0,Alberta,2530047,"Calgary,Edmonton,Red Deer,Lethbridge,Medicine ...",642317.0,55.001251,-115.002136,3.938938
1,British Columbia,3426424,"Vancouver,Victoria,Abbotsford,Kelowna,Nanaimo,...",925186.0,55.001251,-125.002441,3.703497
2,Manitoba,731999,"Winnipeg,Brandon,Thompson,Selkirk,Steinbach,Da...",553556.0,55.001251,-97.001038,1.322358
3,New Brunswick,254834,"Moncton,Saint John,Fredericton,Edmundston,Bath...",71450.0,46.500283,-66.750183,3.566606
4,Newfoundland and Labrador,185120,"Saint Johns,Corner Brook,Happy Valley,Stephenv...",373872.0,53.821733,-61.229553,0.495143
5,Northwest Territories,32623,"Yellowknife,Hay River,Inuvik,Fort McPherson,No...",1183085.0,65.000000,-118.000000,0.027575
6,Nova Scotia,456709,"Halifax,Sydney,New Glasgow,Amherst,Yarmouth,An...",53338.0,45.196040,-63.165379,8.562545
7,Nunavut,26766,"Iqaluit,Rankin Inlet,Arviat,Igloolik,Baker Lak...",1936113.0,70.000007,-90.000000,0.013825
8,Ontario,9788475,"Toronto,Ottawa,Hamilton,Oshawa,Kitchener,Londo...",917741.0,50.000678,-86.000977,10.665836
9,Prince Edward Island,42402,Charlottetown,5660.0,46.335551,-63.146668,7.491519


In [15]:
# Lets visualize the province of Canda using folium map
for lat, lng, label in zip(df_ca.Latitude, df_ca.Longitude, df_ca.province):
    folium.features.Marker(
    [lat, lng],
    popup = label,
    ).add_to(map_ca)
    
# display map
map_ca

In [16]:
# Define foursquare credentials and version
client_id = 'AUDP4MRVSHCP2RDWBPDWJSOSBUFMHCYPOMHY0W02ESKEBM4D'
client_secret = 'QRDC34SWN0ZARTH3I4MBOKPNIOYIPUAOYR01BILPQID3AIS1'
version = '20180604'
limit = 100
query = 'restaurant'

In [17]:
# Find the Province with population density greater than 5
df_pop_den = df_ca[df_ca['population density(Per Sq km)'] >= 5]
df_pop_den = df_pop_den.reset_index(drop=True)
df_pop_den

,province,population,city,Land Area(km Sq),Latitude,Longitude,population density(Per Sq km)
0,Nova Scotia,456709,"Halifax,Sydney,New Glasgow,Amherst,Yarmouth,An...",53338.0,45.196040,-63.165379,8.562545
1,Ontario,9788475,"Toronto,Ottawa,Hamilton,Oshawa,Kitchener,Londo...",917741.0,50.000678,-86.000977,10.665836
2,Prince Edward Island,42402,Charlottetown,5660.0,46.335551,-63.146668,7.491519


In [18]:
# Search for Indian Restaurant at Province Nova Scotia
lat1 = df_pop_den['Latitude'].iloc[0]
lng1 = df_pop_den['Longitude'].iloc[0]
rad1 = df_pop_den['Land Area(km Sq)'].iloc[0]
url1 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(client_id, client_secret, lat1, lng1, version, query, rad1, limit)
url1

'https://api.foursquare.com/v2/venues/search?client_id=AUDP4MRVSHCP2RDWBPDWJSOSBUFMHCYPOMHY0W02ESKEBM4D&client_secret=QRDC34SWN0ZARTH3I4MBOKPNIOYIPUAOYR01BILPQID3AIS1&ll=45.1960403,-63.1653789&v=20180604&query=restaurant&radius=53338.0&limit=100'

In [19]:
# Get the Json request for First province
result_NS = requests.get(url1).json()
result_NS

{'meta': {'code': 200, 'requestId': '5c42c56edb04f57d92bbe1d2'},
 'response': {'venues': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/pizza_',
       'suffix': '.png'},
      'id': '4bf58dd8d48988d1ca941735',
      'name': 'Pizza Place',
      'pluralName': 'Pizza Places',
      'primary': True,
      'shortName': 'Pizza'}],
    'hasPerk': False,
    'id': '4e3842ebd22d4cf95ee955a2',
    'location': {'cc': 'CA',
     'country': 'Canada',
     'distance': 10797,
     'formattedAddress': ['Canada'],
     'labeledLatLngs': [{'label': 'display',
       'lat': 45.254663,
       'lng': -63.275093}],
     'lat': 45.254663,
     'lng': -63.275093},
    'name': "Jake's Pizza & Restaurant",
    'referralId': 'v-1547879790'},
   {'categories': [],
    'hasPerk': False,
    'id': '4dde4b29b0fbc2c4eeff1b34',
    'location': {'cc': 'CA',
     'city': 'Middle Musquodoboit',
     'country': 'Canada',
     'distance': 16725,
     'formattedAddress': ['Middle Musquod

In [20]:
# Transform the json request to pandas dataframe
from pandas.io.json import json_normalize

venues_NS = result_NS['response']['venues']
df_NS = json_normalize(venues_NS)
df_NS

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'name': 'Pizza Place', 'id': '4bf58dd8d48988...",False,4e3842ebd22d4cf95ee955a2,NaN,CA,NaN,Canada,NaN,10797,[Canada],"[{'label': 'display', 'lng': -63.275093, 'lat'...",45.254663,-63.275093,NaN,NaN,Jake's Pizza & Restaurant,v-1547879790
1,[],False,4dde4b29b0fbc2c4eeff1b34,NaN,CA,Middle Musquodoboit,Canada,NaN,16725,"[Middle Musquodoboit NS, Canada]","[{'label': 'display', 'lng': -63.1444130830006...",45.046522,-63.144413,NaN,NS,Reid's Restaurant,v-1547879790
2,"[{'name': 'Thai Restaurant', 'id': '4bf58dd8d4...",False,55167719498ee39ec2616876,165 Willow St,CA,Truro,Canada,NaN,20669,"[165 Willow St, Truro NS B2N 4Z9, Canada]","[{'label': 'display', 'lng': -63.2900905609130...",45.359669,-63.290091,B2N 4Z9,NS,Parichat's Asia Thai Restaurant,v-1547879790
3,"[{'name': 'Restaurant', 'id': '4bf58dd8d48988d...",False,4ce7056be1eeb60c3c559aae,39 Main St,CA,Truro,Canada,Riverside Ave.,21029,"[39 Main St (Riverside Ave.), Truro NS B2N 4G5...","[{'label': 'display', 'lng': -63.271692, 'lat'...",45.369514,-63.271692,B2N 4G5,NS,King Lam Restaurant,v-1547879790
4,"[{'name': 'Fast Food Restaurant', 'id': '4bf58...",False,5911fb7816ef674f7012306f,945 Prince St,CA,Truro,Canada,NaN,20556,"[945 Prince St, Truro NS B2N 1H6, Canada]","[{'label': 'display', 'lng': -63.2730976, 'lat...",45.364434,-63.273098,B2N 1H6,NS,DQ Grill & Chill Restaurant,v-1547879790
5,"[{'name': 'Restaurant', 'id': '4bf58dd8d48988d...",False,59e8d5b7b2958f5a4f38cbf5,86 Connector Rd,CA,Truro,Canada,NaN,20559,"[86 Connector Rd, Truro NS B2N 5B6, Canada]","[{'label': 'display', 'lng': -63.310861, 'lat'...",45.349756,-63.310861,B2N 5B6,NS,Truro Big Stop Restaurant,v-1547879790
6,"[{'name': 'Restaurant', 'id': '4bf58dd8d48988d...",False,4c8ba8629ef0224b9f2f667b,344 Prince St,CA,Truro,Canada,NaN,21604,"[344 Prince St, Truro NS B2N 1E2, Canada]","[{'label': 'display', 'lng': -63.296452, 'lat'...",45.366808,-63.296452,B2N 1E2,NS,Chow Family Restaurant,v-1547879790
7,"[{'name': 'Restaurant', 'id': '4bf58dd8d48988d...",False,53dce823498e6a427f2eaaa9,NaN,CA,Shubenacadie,Canada,NaN,22685,"[Shubenacadie NS, Canada]","[{'label': 'display', 'lng': -63.408585, 'lat'...",45.086041,-63.408585,NaN,NS,Tidal Side Restaurant,v-1547879790
8,"[{'name': 'Seafood Restaurant', 'id': '4bf58dd...",False,4e0894a86284d9ee92dc12ca,25 Treaty Trail,CA,Millbrook,Canada,NaN,18761,"[25 Treaty Trail, Millbrook NS B6L 1W3, Canada]","[{'label': 'display', 'lng': -63.3041266057181...",45.333399,-63.304127,B6L 1W3,NS,Saltscapes Restaurant & General Store,v-1547879790
9,"[{'name': 'Restaurant', 'id': '4bf58dd8d48988d...",False,513f8ccde4b0b797dd7b91b1,Main st,CA,Shubenacadie,Canada,NaN,22697,"[Main st, Shubenacadie NS, Canada]","[{'label': 'display', 'lng': -63.4087311434332...",45.085999,-63.408731,NaN,NS,Tidal Side Restaurant formly Jack's Ship,v-1547879790


In [21]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in df_NS.columns if col.startswith('location.')] + ['id']
df_NS_filtered = df_NS.loc[:, filtered_columns]
df_NS_filtered

,name,categories,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,id
0,Jake's Pizza & Restaurant,"[{'name': 'Pizza Place', 'id': '4bf58dd8d48988...",NaN,CA,NaN,Canada,NaN,10797,[Canada],"[{'label': 'display', 'lng': -63.275093, 'lat'...",45.254663,-63.275093,NaN,NaN,4e3842ebd22d4cf95ee955a2
1,Reid's Restaurant,[],NaN,CA,Middle Musquodoboit,Canada,NaN,16725,"[Middle Musquodoboit NS, Canada]","[{'label': 'display', 'lng': -63.1444130830006...",45.046522,-63.144413,NaN,NS,4dde4b29b0fbc2c4eeff1b34
2,Parichat's Asia Thai Restaurant,"[{'name': 'Thai Restaurant', 'id': '4bf58dd8d4...",165 Willow St,CA,Truro,Canada,NaN,20669,"[165 Willow St, Truro NS B2N 4Z9, Canada]","[{'label': 'display', 'lng': -63.2900905609130...",45.359669,-63.290091,B2N 4Z9,NS,55167719498ee39ec2616876
3,King Lam Restaurant,"[{'name': 'Restaurant', 'id': '4bf58dd8d48988d...",39 Main St,CA,Truro,Canada,Riverside Ave.,21029,"[39 Main St (Riverside Ave.), Truro NS B2N 4G5...","[{'label': 'display', 'lng': -63.271692, 'lat'...",45.369514,-63.271692,B2N 4G5,NS,4ce7056be1eeb60c3c559aae
4,DQ Grill & Chill Restaurant,"[{'name': 'Fast Food Restaurant', 'id': '4bf58...",945 Prince St,CA,Truro,Canada,NaN,20556,"[945 Prince St, Truro NS B2N 1H6, Canada]","[{'label': 'display', 'lng': -63.2730976, 'lat...",45.364434,-63.273098,B2N 1H6,NS,5911fb7816ef674f7012306f
5,Truro Big Stop Restaurant,"[{'name': 'Restaurant', 'id': '4bf58dd8d48988d...",86 Connector Rd,CA,Truro,Canada,NaN,20559,"[86 Connector Rd, Truro NS B2N 5B6, Canada]","[{'label': 'display', 'lng': -63.310861, 'lat'...",45.349756,-63.310861,B2N 5B6,NS,59e8d5b7b2958f5a4f38cbf5
6,Chow Family Restaurant,"[{'name': 'Restaurant', 'id': '4bf58dd8d48988d...",344 Prince St,CA,Truro,Canada,NaN,21604,"[344 Prince St, Truro NS B2N 1E2, Canada]","[{'label': 'display', 'lng': -63.296452, 'lat'...",45.366808,-63.296452,B2N 1E2,NS,4c8ba8629ef0224b9f2f667b
7,Tidal Side Restaurant,"[{'name': 'Restaurant', 'id': '4bf58dd8d48988d...",NaN,CA,Shubenacadie,Canada,NaN,22685,"[Shubenacadie NS, Canada]","[{'label': 'display', 'lng': -63.408585, 'lat'...",45.086041,-63.408585,NaN,NS,53dce823498e6a427f2eaaa9
8,Saltscapes Restaurant & General Store,"[{'name': 'Seafood Restaurant', 'id': '4bf58dd...",25 Treaty Trail,CA,Millbrook,Canada,NaN,18761,"[25 Treaty Trail, Millbrook NS B6L 1W3, Canada]","[{'label': 'display', 'lng': -63.3041266057181...",45.333399,-63.304127,B6L 1W3,NS,4e0894a86284d9ee92dc12ca
9,Tidal Side Restaurant formly Jack's Ship,"[{'name': 'Restaurant', 'id': '4bf58dd8d48988d...",Main st,CA,Shubenacadie,Canada,NaN,22697,"[Main st, Shubenacadie NS, Canada]","[{'label': 'display', 'lng': -63.4087311434332...",45.085999,-63.408731,NaN,NS,513f8ccde4b0b797dd7b91b1


In [22]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [23]:
# filter the category for each row
df_NS_filtered['categories'] = df_NS_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
df_NS_filtered.columns = [column.split('.')[-1] for column in df_NS_filtered.columns]
df_NS_filtered = (df_NS_filtered[df_NS_filtered['categories'] == 'Restaurant']).reset_index(drop=True)
df_NS_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,King Lam Restaurant,Restaurant,39 Main St,CA,Truro,Canada,Riverside Ave.,21029,"[39 Main St (Riverside Ave.), Truro NS B2N 4G5...","[{'label': 'display', 'lng': -63.271692, 'lat'...",45.369514,-63.271692,B2N 4G5,NS,4ce7056be1eeb60c3c559aae
1,Truro Big Stop Restaurant,Restaurant,86 Connector Rd,CA,Truro,Canada,NaN,20559,"[86 Connector Rd, Truro NS B2N 5B6, Canada]","[{'label': 'display', 'lng': -63.310861, 'lat'...",45.349756,-63.310861,B2N 5B6,NS,59e8d5b7b2958f5a4f38cbf5
2,Chow Family Restaurant,Restaurant,344 Prince St,CA,Truro,Canada,NaN,21604,"[344 Prince St, Truro NS B2N 1E2, Canada]","[{'label': 'display', 'lng': -63.296452, 'lat'...",45.366808,-63.296452,B2N 1E2,NS,4c8ba8629ef0224b9f2f667b
3,Tidal Side Restaurant,Restaurant,NaN,CA,Shubenacadie,Canada,NaN,22685,"[Shubenacadie NS, Canada]","[{'label': 'display', 'lng': -63.408585, 'lat'...",45.086041,-63.408585,NaN,NS,53dce823498e6a427f2eaaa9
4,Tidal Side Restaurant formly Jack's Ship,Restaurant,Main st,CA,Shubenacadie,Canada,NaN,22697,"[Main st, Shubenacadie NS, Canada]","[{'label': 'display', 'lng': -63.4087311434332...",45.085999,-63.408731,NaN,NS,513f8ccde4b0b797dd7b91b1
5,Enfield Big Stop Restaurant,Restaurant,6757 Highway 2,CA,Enfield,Canada,NaN,41745,"[6757 Highway 2, Enfield NS B2T 1C8, Canada]","[{'label': 'display', 'lng': -63.535959, 'lat'...",44.927493,-63.535959,B2T 1C8,NS,59e8d5bc04d1ae3faaf2965a
6,Hilton Garden Inn Restaurant,Restaurant,200 Pratt And Whitney DR,CA,Halifax,Canada,NaN,47675,"[200 Pratt And Whitney DR, Halifax NS, Canada]","[{'label': 'display', 'lng': -63.5446139773690...",44.861995,-63.544614,NaN,NS,507364d8e4b035a2183d09f7
7,glooscap restaurant,Restaurant,758 upper main street,CA,Parrsboro,Canada,NaN,55195,"[758 upper main street, Parrsboro NS, Canada]","[{'label': 'display', 'lng': -63.8079757690429...",45.399845,-63.807976,NaN,NS,5197a526498ea88ef490d5e7
8,Big Al's Family Restaurant & Lounge,Restaurant,9 Station Road,CA,Tatamagouche,Canada,NaN,57845,"[9 Station Road, Tatamagouche NS B0K 1V0, Canada]","[{'label': 'display', 'lng': -63.285274, 'lat'...",45.708830,-63.285274,B0K 1V0,NS,4bd3042f9854d13a27a9fc4d


In [24]:
# Search for Indian Restaurant at Province Ontario
lat2 = df_pop_den['Latitude'].iloc[1]
lng2 = df_pop_den['Longitude'].iloc[1]
rad2 = df_pop_den['Land Area(km Sq)'].iloc[1]
url2 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(client_id, client_secret, lat2, lng2, version, query, rad2, limit)
url2

'https://api.foursquare.com/v2/venues/search?client_id=AUDP4MRVSHCP2RDWBPDWJSOSBUFMHCYPOMHY0W02ESKEBM4D&client_secret=QRDC34SWN0ZARTH3I4MBOKPNIOYIPUAOYR01BILPQID3AIS1&ll=50.000678,-86.000977&v=20180604&query=restaurant&radius=917741.0&limit=100'

In [25]:
res2 = requests.get(url2).json()
res2

{'meta': {'code': 200, 'requestId': '5c42c56e4434b947c555f299'},
 'response': {'venues': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_',
       'suffix': '.png'},
      'id': '52e81612bcbc57f1066b7a00',
      'name': 'Comfort Food Restaurant',
      'pluralName': 'Comfort Food Restaurants',
      'primary': True,
      'shortName': 'Comfort Food'}],
    'hasPerk': False,
    'id': '546755ef498e6a5dab6ff997',
    'location': {'cc': 'CA',
     'country': 'Canada',
     'distance': 45292,
     'formattedAddress': ['Canada'],
     'labeledLatLngs': [{'label': 'display',
       'lat': 49.78154373168945,
       'lng': -86.5331039428711}],
     'lat': 49.78154373168945,
     'lng': -86.5331039428711},
    'name': 'Woodlands Restaurant',
    'referralId': 'v-1547879790'},
   {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/asian_',
       'suffix': '.png'},
      'id': '4bf58dd8d48988d145941735',
      'name': 'Chines

In [26]:
venues_Ont = res2['response']['venues']

df_Ont = json_normalize(venues_Ont)
df_Ont

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.state,name,referralId
0,"[{'name': 'Comfort Food Restaurant', 'id': '52...",False,546755ef498e6a5dab6ff997,NaN,CA,NaN,Canada,45292,[Canada],"[{'label': 'display', 'lng': -86.5331039428711...",49.781544,-86.533104,NaN,Woodlands Restaurant,v-1547879790
1,"[{'name': 'Chinese Restaurant', 'id': '4bf58dd...",False,4e14e464b0fb1946c78f01d2,NaN,CA,Greenstone,Canada,74652,"[Greenstone ON, Canada]","[{'label': 'display', 'lng': -86.9488451383074...",49.724276,-86.948845,ON,Red Dragon Chinese Restaurant,v-1547879790
2,"[{'name': 'Breakfast Spot', 'id': '4bf58dd8d48...",False,520e60d511d21c54f47a9ccf,NaN,CA,NaN,Canada,98073,[Canada],"[{'label': 'display', 'lng': -85.827577, 'lat'...",49.126872,-85.827577,NaN,Rendezvous Restaurant.,v-1547879790
3,"[{'name': 'Restaurant', 'id': '4bf58dd8d48988d...",False,4efb476be5fa4f286ec8e6c0,NaN,CA,NaN,Canada,98199,[Canada],"[{'label': 'display', 'lng': -85.83934, 'lat':...",49.124780,-85.839340,NaN,Rendezvous restaurant,v-1547879790
4,[],False,4cc5b23bb2beb1f74d3f244c,54 Huron Walk,CA,Manitouwadge,Canada,98043,"[54 Huron Walk, Manitouwadge ON, Canada]","[{'label': 'display', 'lng': -85.8279820860184...",49.127110,-85.827982,ON,Capri Restaurant,v-1547879790
5,"[{'name': 'American Restaurant', 'id': '4bf58d...",False,4fa1d6c7e4b0e0605d9e87af,Highway 17,CA,Marathon,Canada,140262,"[Highway 17, Marathon ON, Canada]","[{'label': 'display', 'lng': -86.361751, 'lat'...",48.762748,-86.361751,ON,Marino's Restaurant and Lounge,v-1547879790


In [27]:
filtered_columns = ['name', 'categories'] + [col for col in df_Ont.columns if col.startswith('location')] + ['id']
df_Ont_filtered = df_Ont.loc[:, filtered_columns]
df_Ont_filtered

,name,categories,location.address,location.cc,location.city,location.country,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.state,id
0,Woodlands Restaurant,"[{'name': 'Comfort Food Restaurant', 'id': '52...",NaN,CA,NaN,Canada,45292,[Canada],"[{'label': 'display', 'lng': -86.5331039428711...",49.781544,-86.533104,NaN,546755ef498e6a5dab6ff997
1,Red Dragon Chinese Restaurant,"[{'name': 'Chinese Restaurant', 'id': '4bf58dd...",NaN,CA,Greenstone,Canada,74652,"[Greenstone ON, Canada]","[{'label': 'display', 'lng': -86.9488451383074...",49.724276,-86.948845,ON,4e14e464b0fb1946c78f01d2
2,Rendezvous Restaurant.,"[{'name': 'Breakfast Spot', 'id': '4bf58dd8d48...",NaN,CA,NaN,Canada,98073,[Canada],"[{'label': 'display', 'lng': -85.827577, 'lat'...",49.126872,-85.827577,NaN,520e60d511d21c54f47a9ccf
3,Rendezvous restaurant,"[{'name': 'Restaurant', 'id': '4bf58dd8d48988d...",NaN,CA,NaN,Canada,98199,[Canada],"[{'label': 'display', 'lng': -85.83934, 'lat':...",49.124780,-85.839340,NaN,4efb476be5fa4f286ec8e6c0
4,Capri Restaurant,[],54 Huron Walk,CA,Manitouwadge,Canada,98043,"[54 Huron Walk, Manitouwadge ON, Canada]","[{'label': 'display', 'lng': -85.8279820860184...",49.127110,-85.827982,ON,4cc5b23bb2beb1f74d3f244c
5,Marino's Restaurant and Lounge,"[{'name': 'American Restaurant', 'id': '4bf58d...",Highway 17,CA,Marathon,Canada,140262,"[Highway 17, Marathon ON, Canada]","[{'label': 'display', 'lng': -86.361751, 'lat'...",48.762748,-86.361751,ON,4fa1d6c7e4b0e0605d9e87af


In [28]:
# Filter the category for each row
df_Ont_filtered['categories'] = df_Ont_filtered.apply(get_category_type, axis=1)
df_Ont_filtered.columns = [column.split('.')[-1] for column in df_Ont_filtered.columns]
df_Ont_filtered

,name,categories,address,cc,city,country,distance,formattedAddress,labeledLatLngs,lat,lng,state,id
0,Woodlands Restaurant,Comfort Food Restaurant,NaN,CA,NaN,Canada,45292,[Canada],"[{'label': 'display', 'lng': -86.5331039428711...",49.781544,-86.533104,NaN,546755ef498e6a5dab6ff997
1,Red Dragon Chinese Restaurant,Chinese Restaurant,NaN,CA,Greenstone,Canada,74652,"[Greenstone ON, Canada]","[{'label': 'display', 'lng': -86.9488451383074...",49.724276,-86.948845,ON,4e14e464b0fb1946c78f01d2
2,Rendezvous Restaurant.,Breakfast Spot,NaN,CA,NaN,Canada,98073,[Canada],"[{'label': 'display', 'lng': -85.827577, 'lat'...",49.126872,-85.827577,NaN,520e60d511d21c54f47a9ccf
3,Rendezvous restaurant,Restaurant,NaN,CA,NaN,Canada,98199,[Canada],"[{'label': 'display', 'lng': -85.83934, 'lat':...",49.124780,-85.839340,NaN,4efb476be5fa4f286ec8e6c0
4,Capri Restaurant,None,54 Huron Walk,CA,Manitouwadge,Canada,98043,"[54 Huron Walk, Manitouwadge ON, Canada]","[{'label': 'display', 'lng': -85.8279820860184...",49.127110,-85.827982,ON,4cc5b23bb2beb1f74d3f244c
5,Marino's Restaurant and Lounge,American Restaurant,Highway 17,CA,Marathon,Canada,140262,"[Highway 17, Marathon ON, Canada]","[{'label': 'display', 'lng': -86.361751, 'lat'...",48.762748,-86.361751,ON,4fa1d6c7e4b0e0605d9e87af


In [29]:
# Search for Restaurant at Province Prince Edward Island
query = 'indian'
lat3 = df_pop_den['Latitude'].iloc[2]
lng3 = df_pop_den['Longitude'].iloc[2]
rad3 = df_pop_den['Land Area(km Sq)'].iloc[2]
url3 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(client_id, client_secret, lat3, lng3, version, query, rad3, limit)
url3

'https://api.foursquare.com/v2/venues/search?client_id=AUDP4MRVSHCP2RDWBPDWJSOSBUFMHCYPOMHY0W02ESKEBM4D&client_secret=QRDC34SWN0ZARTH3I4MBOKPNIOYIPUAOYR01BILPQID3AIS1&ll=46.3355508,-63.1466676&v=20180604&query=indian&radius=5660.0&limit=100'

In [30]:
# Get the request response
res3 = requests.get(url3).json()
res3

{'meta': {'code': 200, 'requestId': '5c42c56f6a607129e2c7a02f'},
 'response': {'venues': []}}

In [31]:
# get the venue details from json file
venues_PE = res3['response']['venues']

# convert json to dataframe
df_PE = json_normalize(venues_PE)
df_PE

""


In [32]:
# Lets get the province Nova Scotia
for name, city in zip(df_NS_filtered.name, df_NS_filtered.city):
    print('Restaurant Name: {} Location: {}'.format(name, city))

Restaurant Name: King Lam Restaurant Location: Truro
Restaurant Name: Truro Big Stop Restaurant Location: Truro
Restaurant Name: Chow Family Restaurant Location: Truro
Restaurant Name: Tidal Side Restaurant Location: Shubenacadie
Restaurant Name: Tidal Side Restaurant formly Jack's Ship Location: Shubenacadie
Restaurant Name: Enfield Big Stop Restaurant Location: Enfield
Restaurant Name: Hilton Garden Inn Restaurant Location: Halifax
Restaurant Name: glooscap restaurant Location: Parrsboro
Restaurant Name: Big Al's Family Restaurant & Lounge Location: Tatamagouche


In [33]:
# Lets get the province Ontario
for name, city in zip(df_Ont_filtered.name, df_Ont_filtered.city):
    print('Restaurant Name: {} Location: {}'.format(name, city))

Restaurant Name: Woodlands Restaurant Location: nan
Restaurant Name: Red Dragon Chinese Restaurant Location: Greenstone
Restaurant Name: Rendezvous Restaurant. Location: nan
Restaurant Name: Rendezvous restaurant Location: nan
Restaurant Name: Capri Restaurant Location: Manitouwadge
Restaurant Name: Marino's Restaurant and Lounge Location: Marathon


In [34]:
df_pop_den

,province,population,city,Land Area(km Sq),Latitude,Longitude,population density(Per Sq km)
0,Nova Scotia,456709,"Halifax,Sydney,New Glasgow,Amherst,Yarmouth,An...",53338.0,45.196040,-63.165379,8.562545
1,Ontario,9788475,"Toronto,Ottawa,Hamilton,Oshawa,Kitchener,Londo...",917741.0,50.000678,-86.000977,10.665836
2,Prince Edward Island,42402,Charlottetown,5660.0,46.335551,-63.146668,7.491519


In [46]:
# Check for the cityin province Nova Scotia which has the restaurant
for city1 in df_NS_filtered['city']:
    for city2 in df_pop_den['city'].iloc[0].split(','):
        if city1 == city2:
            print (city1)
            print (df_NS_filtered[df_NS_filtered['city']==city1]['name'])

Halifax
6    Hilton Garden Inn Restaurant
Name: name, dtype: object


In [47]:
# Check for the cityin province Ontario which has the restaurant
for city1 in df_Ont_filtered['city']:
    for city2 in df_pop_den['city'].iloc[0].split(','):
        if city1 == city2:
            print (city1)
            print (df_Ont_filtered[df_Ont_filtered['city']==city1]['name'])